## IBM Coursera Data Science Capstone Project Week 3

## Question 1

### First import the necessary libraries

In [1]:
# used to make tab autocomplete work faster 
%config Completer.use_jedi = False

In [18]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [19]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

**Get the html contents in string format**

In [20]:
data = requests.get(url).text
# data

**Convert data to a Soup Object**

In [21]:
soup = BeautifulSoup(data, 'html.parser')
# soup

In [22]:
table = soup.find('table')
table_contents = []
for i,row in enumerate(table.findAll('td')):
    cell = {}
    if 'not assigned' in row.span.text.lower():
        pass
    else:
#         df_torneigh.iloc[i,'PostalCode'] = row.text[:3]
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = row.span.text.split('(')[0]
        cell['Neighborhood'] = row.span.text.split('(')[1].replace(' /',',').strip(')')
        table_contents.append(cell)
df = pd.DataFrame(table_contents)

In [23]:
df[df['Borough'].str.len() > 20]

,PostalCode,Borough,Neighborhood
35,M4J,East YorkEast Toronto,The Danforth East
76,M7R,MississaugaCanada Post Gateway Processing Centre,Enclave of L4W
92,M5W,Downtown TorontoStn A PO Boxes25 The Esplanade,Enclave of M5E
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L


In [24]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [25]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


**Shape of the dataframe**

In [26]:
df.shape

(103, 3)

## Question 2

**Try to get the location data using geocoder**

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
# while(lat_lng_coords is None):
#     g = geocoder.google('{}, Toronto, Ontario'.format('M7A'))
#     lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

**Geocoder module did not work. So get the data from the provided csv**

In [27]:
loc_df = pd.read_csv('Geospatial_Coordinates.csv')
loc_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Merge the two dataframes**

In [29]:
df = pd.merge(df,loc_df,left_on='PostalCode',right_on='Postal Code')
df.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


**Drop the duplicate Postal Code column**

In [32]:
df.drop('Postal Code',axis=1,inplace=True)

**Question 2 final result:**

In [33]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
